In [1]:
# !!pip install guardrails-ai

In [2]:
# !!pip install sqlvalidator

## Imports above required

In [1]:
from langchain.agents import create_pandas_dataframe_agent
from langchain.chat_models import ChatOpenAI
import os,openai
from langchain.llms import OpenAI
import pandas as pd
from langchain import PromptTemplate


In [3]:
# open_ai_key = '***************'
os.environ['OPENAI_API_KEY'] = open_ai_key
openai.api_key = open_ai_key

In [8]:

def get_completion(prompt, model="gpt-3.5-turbo",temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [9]:
prompt = PromptTemplate.from_template(template = '''Given a user query {query} now generate a sql query for it.
Give a JSON repsonse''')

In [10]:
prompt = prompt.format(query = 'Select the name of the employee who has the highest salary.')

In [11]:
result = get_completion(prompt=prompt,temperature=0.0)

print("Answer:",result,"\n")

Answer: SQL Query:

SELECT name
FROM employees
WHERE salary = (SELECT MAX(salary) FROM employees);

JSON Response:

{
  "name": "John Doe"
} 



## With Guardrails


### Defining Rail spec.

In [12]:
rail_str = """
<rail version="0.1">
<output>
<string description="Generate SQL for the given natural language instruction." format="bug-free-sql" name="generated_sql" on-fail-bug-free-sql="reask"></string>
</output>
<prompt>

Generate a valid SQL query for the following natural language instruction:

${nl_instruction}

${gr.complete_json_suffix}
</prompt>
</rail>
"""

In [13]:
from guardrails.validators import BugFreeSQL
from pydantic import BaseModel, Field

prompt = """

Generate a valid SQL query for the following natural language instruction:

${nl_instruction}

${gr.complete_json_suffix}
"""


## defining a pydantic basemodel
class ValidSql(BaseModel):
    generated_sql: str = Field(description="Generate SQL for the given natural language instruction.", validators=[BugFreeSQL(on_fail="reask")])

In [14]:
import guardrails as gd

from rich import print

In [15]:
guard = gd.Guard.from_rail_string(rail_str)


In [16]:
guard = gd.Guard.from_pydantic(output_class=ValidSql, prompt=prompt)

In [17]:
print(guard.base_prompt)

Generate a valid SQL query for the following natural language instruction:

${nl_instruction}


Given below is XML that describes the information to extract from this document and the tags to extract it into.

<output>
    <string name="generated_sql" format="bug-free-sql" description="Generate SQL for the given natural language 
instruction."/>
</output>


ONLY return a valid JSON object (no other text is necessary), where the key of the field in JSON is the `name` 
attribute of the corresponding XML, and the value is of the type specified by the corresponding XML's tag. The JSON
MUST conform to the XML format, including any types and format requests e.g. requests for lists, objects and 
specific types. Be correct and concise. If you are unsure anywhere, enter `null`.

Here are examples of simple (XML, JSON) pairs that show the expected behavior:
- `<string name='foo' format='two-words lower-case' />` => `{'foo': 'example one'}`
- `<list name='bar'><string format='upper-case' /></list>` => `{"bar": ['STRING ONE', 'STRING TWO', etc.]}`
- `<object name='baz'><string name="foo" format="capitalize two-words" /><integer name="index" format="1-indexed" 
/></object>` => `{'baz': {'foo': 'Some String', 'index': 1}}`

In [18]:
import openai

raw_llm_response, validated_response = guard(
    openai.Completion.create,
    prompt_params={
        "nl_instruction": "Select the name of the employee who has the highest salary."
    },
    engine="text-davinci-003",
    max_tokens=2048,
    temperature=0,
)


Async event loop found, but guard was invoked synchronously.For validator parallelization, please call `validate_async` instead.


In [19]:
print(validated_response)

{'generated_sql': 'SELECT name FROM employee ORDER BY salary DESC LIMIT 1'}

In [20]:
print(guard.state.most_recent_call.tree)

Logs
└── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
    │ ╭──────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────╮ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ │ Generate a valid SQL query for the following natural language instruction:                              │ │
    │ │                                                                                                         │ │
    │ │ Select the name of the employee who has the highest salary.                                             │ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ │ Given below is XML that describes the information to extract from this document and the tags to extract │ │
    │ │ it into.                                                                                                │ │
    │ │                                                                                                         │ │
    │ │ <output>                                                                                                │ │
    │ │     <string name="generated_sql" format="bug-free-sql" description="Generate SQL for the given natural  │ │
    │ │ language instruction."/>                                                                                │ │
    │ │ </output>                                                                                               │ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ │ ONLY return a valid JSON object (no other text is necessary), where the key of the field in JSON is the │ │
    │ │ `name` attribute of the corresponding XML, and the value is of the type specified by the corresponding  │ │
    │ │ XML's tag. The JSON MUST conform to the XML format, including any types and format requests e.g.        │ │
    │ │ requests for lists, objects and specific types. Be correct and concise. If you are unsure anywhere,     │ │
    │ │ enter `null`.                                                                                           │ │
    │ │                                                                                                         │ │
    │ │ Here are examples of simple (XML, JSON) pairs that show the expected behavior:                          │ │
    │ │ - `<string name='foo' format='two-words lower-case' />` => `{'foo': 'example one'}`                     │ │
    │ │ - `<list name='bar'><string format='upper-case' /></list>` => `{"bar": ['STRING ONE', 'STRING TWO',     │ │
    │ │ etc.]}`                                                                                                 │ │
    │ │ - `<object name='baz'><string name="foo" format="capitalize two-words" /><integer name="index"          │ │
    │ │ format="1-indexed" /></object>` => `{'baz': {'foo': 'Some String', 'index': 1}}`                        │ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ │ Json Output:                                                                                            │ │
    │ │                                            